## <b><font color='darkblue'>Building with Agent Development Kit — Workshop Guide</font></b>
([source](https://medium.com/@jereljohnvelarde/building-with-agent-development-kit-workshop-guide-32047e6a41ae)) <font size='3ptx'><b>Google released the Agent development kit’s preview version around a month ago and just released v1.0 at Google I/O’25 (May 20, 2025)</b> So I think we just hosted the first Agent Development Workshop...</font>

Here’s some key highlights of the event:
* **Workshop 1**: Building your first agent with ADK
* **Workshop 2**: Function Calling and Intro to Multiagent Systems

In [5]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str, raw_text=False):
    source_code = !cat $src_path
    if raw_text:
        print('\n'.join(source_code))
    else:
        display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

## <b><font color='darkblue'>Walkthrough of Workshop 1</font></b>
([source](https://youtu.be/UalqWbV09bw), [Github](https://github.com/GeneralJerel/ADK-Your-First-Workflow)) What’s included:
* Your First Agent: Automation Analyst
* Your First Agentic Workflow: Role Automation workflow


**Automation Analyst**
> Takes in a job description and outputs a markdown table analysis of automation potential

**Role Automation Workflow**
> A sequential agent workflow that writes a job description, automation analysis, and write prompts for the highly automateable tasks.

### <b><font color='darkgreen'>Chapter 1: Setting up</font></b>

**Pre-Requisites**
* Python
* IDE of your choice
* Setup `.venv`
* Install ADK (pip install google-adk)

In [1]:
!pip freeze | grep 'google-adk'

google-adk==1.7.0


### <b><font color='darkgreen'>Chapter 2: Your First AI Agent</font></b>
<font size='3ptx'><b>In [ADK](https://google.github.io/adk-docs/agents/), An Agent is a self-contained execution unit designed to act autonomously to achieve specific goals.</b> Agents can perform tasks, interact with users, utilize external tools, and coordinate with other agents.</font>
![agent flow](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*xf0oN2sXoXwpyGrOAVI01w.png)

#### <b>Automation Analyst</b>
Takes in a job description and outputs a markdown table analysis of automation potential.

#### <b>Folder Structure</b>
```
Parent-folder
└─automation_analyst/
  ├── __init__.py
  ├── agent.py          # Main agent configuration
  ├── prompt.py         # Agent instructions and prompt template
  └── .gitignore
```

Your CLI should be on the parent folder
* <b><font color='olive'>sample .env</font></b>
```shell
GOOGLE_GENAI_USE_VERTEXAI=FALSE
GOOGLE_API_KEY=YOUR_GOOGLE_API_KEY
```

* <b><font color='olive'>__init__.py</font></b>
```python
from . import agent
```

* <b><font color='olive'>agent.py</font></b>
```python
from google.adk.agents import Agent

from . import prompt

root_agent = Agent(
    name="automation_analyst",
    model="gemini-2.0-flash",
    description=(
        "Give me a role or a job description and I'll break it down to tasks and analyze for automation potential"
    ),
    instruction=prompt.ROOT_PROMPT,
)
```

* <b><font color='olive'>prompt.py</font></b>
```python
ROOT_PROMPT= """
**Context**
You are a Business Analysis Assistant designed to help operations managers, HR strategists, and transformation consultants analyze job roles and assess which tasks are suitable for automation. Your primary goal is to deconstruct a given job or role into discrete tasks, evaluate each task's automation potential (High, Medium, Low), and provide strategic recommendations for improving efficiency through automation.

**Instructions**
- Accept a plain-language job title or job description as input.
- ...
```

#### <b>Running the Workflow</b>
* Navigate to the CLI for the parent folder
* Run
```shell
$ adk web
...
INFO:     Started server process [2631917]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://localhost:8000.                         |
+-----------------------------------------------------------------------------+

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

* Open brower:
![Web](images/1.png)

![Web](images/2.png)

### <b><font color='darkgreen'>Chapter 3: Your First Workflow</font></b>
![ui](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*pG48ZotfDcN1faYT)

#### <b>Role Automation Workflow</b>
A sequential agent workflow that writes a job description, automation analysis, and write prompts for the highly automateable tasks.
* Job Description Agent
* Automation Analyst
* Prompt Engineer

#### <b>Folder Structure</b>
```
adk-first-workflow/
├── role-automation-workflow/
│   ├── sub_agents/           # Contains specialized sub-agents
│   │   ├── __init__.py      # Exports sub-agents
│   │   ├── agent.py         # Sub-agent implementations
│   │   └── prompt.py        # Sub-agent prompts
│   ├── agent.py            # Main agent implementation
│   ├── prompt.py           # Agent prompts and instructions
│   └── __init__.py         # Package initialization
├── .venv/                   # Python virtual environment
└── .gitignore              # Git ignore file
```

Your CLI should be on the parent folder `adk-first-workflow`. We are going to declare a sequential agentic workflow

#### <b>role-automation-workflow</b>
We are going to declare a sequential agentic workflow:

- **sub_agents/agent.py**
```python
from google.adk.agents.llm_agent import Agent

from . import prompt

# --- 1. Define Sub-Agents for Each Pipeline Stage ---

job_description_agent = Agent(
    model='gemini-2.0-flash',
    name="job_description_agent",
    description="A helpful agent to find job descriptions from google search",
    instruction=prompt.JOB_DESCRIPTION_AGENT_PROMPT,
    output_key="job_description",
)

workflow_analysis_agent = Agent(
    model='gemini-2.0-flash',
    name="workflow_analysis_agent",
    description="A helpful agent to analyze workflows",
    instruction=prompt.WORKFLOW_ANALYSIS_AGENT_PROMPT,
    output_key="workflow_analysis",
)

prompt_engineer_agent = Agent(
    model='gemini-2.0-flash',
    name="prompt_engineer_agent",
    description="selects high potential for optimazation from analysis and creates prompts, and adds them to the prompt library",
    instruction=prompt.PROMPT_ENGINEER_AGENT_PROMPT,
    output_key="prompt_library",
)
```

- **sub_agents/prompt.py**: Setup prompt instructions for the sub agents by setting up prompt.py under sub_agents folder.

In [6]:
show_source_code('adk-first-workflow/role-automation-workflow/sub_agents/prompt.py', raw_text=True)

JOB_DESCRIPTION_AGENT_PROMPT = """
**Context**
You are a global job role expert with deep knowledge of various professional positions across industries. Your expertise allows you to craft detailed, accurate, and realistic job descriptions that reflect current industry standards and best practices.

**Instructions**
- Analyze the user's request to understand the specific role and requirements they need.
- Draw upon your comprehensive knowledge of job roles, industry standards, and organizational structures.
- Create detailed job descriptions that include:
  - Clear role responsibilities
  - Required qualifications and skills
  - Industry-specific requirements
  - Expected experience levels
  - Key performance indicators
- Ensure descriptions are:
  - Realistic and practical
  - Aligned with current industry standards
  - Comprehensive yet concise
  - Well-structured and easy to read

**Response Format**
Return a clean, well-formatted job description with these sections:
- **Job Title**


#### <b>Running the Workflow</b>
* Navigate to the CLI for the parent folder
* run adk web:
```shell
$ pwd
.../ml_articles/medium/Building_with_agent_development_kit_workshop_guide/adk-first-workflow
$ adk web
```

### <b><font color='darkgreen'>Summary</font></b>
By the end of this you should have
* Developed the fundamentals of AI agent development
* Built your first Agent with ADK
* Built your first AI Agentic workflow with ADK